# BitCoin Prediction using Deep Neural Network 

###### Import

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
plt.style.use("ggplot")

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

###### Load and read the data

In [10]:
# Loading/Reading in the Data
df = pd.read_csv("BTC-USD.csv")

# Data Preprocessing
### Setting the datetime index as the date, only selecting the 'Close' column, then only the last 1000 closing prices.
df = df.set_index("Date")[['Close']].tail(1000)
df = df.set_index(pd.to_datetime(df.index))

# Normalizing/Scaling the Data
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

In [12]:

def visualize_training_results(results):
    """
    Plots the loss and accuracy for the training and testing data
    """
    history = results.history
    plt.figure(figsize=(12,4))
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure(figsize=(12,4))
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()
    
    
def split_sequence(seq, n_steps_in, n_steps_out):
    """
    Splits the univariate time sequence
    """
    X, y = [], []
    
    for i in range(len(seq)):
        end = i + n_steps_in
        out_end = end + n_steps_out
        
        if out_end > len(seq):
            break
        
        seq_x, seq_y = seq[i:end], seq[end:out_end]
        
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X), np.array(y)


def layer_maker(n_layers, n_nodes, activation, drop=None, d_rate=.5):
    """
    Create a specified number of hidden layers for an RNN
    Optional: Adds regularization option, dropout layer to prevent potential overfitting if necessary
    """
    
    # Creating the specified number of hidden layers with the specified number of nodes
    for x in range(1,n_layers+1):
        model.add(LSTM(n_nodes, activation=activation, return_sequences=True))

        # Adds a Dropout layer after every Nth hidden layer (the 'drop' variable)
        try:
            if x % drop == 0:
                model.add(Dropout(d_rate))
        except:
            pass

In [14]:
# How many periods looking back to train
n_per_in  = 30

# How many periods ahead to predict
n_per_out = 10

# Features (in this case it's 1 because there is only one feature: price)
n_features = 1

# Splitting the data into appropriate sequences
X, y = split_sequence(list(df.Close), n_per_in, n_per_out)

# Reshaping the X variable from 2D to 3D
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [15]:
# Instantiating the model
model = Sequential()

# Activation
activ = "softsign"

# Input layer
model.add(LSTM(30, activation=activ, return_sequences=True, input_shape=(n_per_in, n_features)))

# Hidden layers
layer_maker(n_layers=6, n_nodes=12, activation=activ)

# Final Hidden layer
model.add(LSTM(10, activation=activ))

# Output layer
model.add(Dense(n_per_out))

# Model summary
model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 30)            3840      
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 12)            2064      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_6 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 12)            1200      
_______

In [18]:
model.compile(loss='mean_squared_error', optimizer="Adam", metrics=['accuracy'])

In [ ]:
res = model.fit(X, y, epochs=800, batch_size=32, validation_split=0.1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 864 samples, validate on 97 samples
Epoch 1/800
864/864 [==============================] - 14s 16ms/step - loss: 0.0982 - acc: 0.1331 - val_loss: 0.0947 - val_acc: 0.3402
Epoch 2/800
864/864 [==============================] - 4s 5ms/step - loss: 0.0506 - acc: 0.1551 - val_loss: 0.0288 - val_acc: 0.3402
Epoch 3/800
864/864 [==============================] - 5s 5ms/step - loss: 0.0349 - acc: 0.0903 - val_loss: 0.0193 - val_acc: 0.0619
Epoch 4/800
864/864 [==============================] - 5s 5ms/step - loss: 0.0267 - acc: 0.1019 - val_loss: 0.0040 - val_acc: 0.0515
Epoch 5/800
864/864 [==============================] - 5s 6ms/step - loss: 0.0155 - acc: 0.0961 - val_loss: 0.0030 - val_acc: 0.0619
Epoch 6/800
864/864 [==============================] - 5s 5ms/step - loss: 0.0129 - acc: 0.1088 - val_loss: 0.0061 - val_acc: 0.1753
Epoch 7/800
864/864 [==============================] - 5s 6

864/864 [==============================] - 5s 6ms/step - loss: 0.0031 - acc: 0.1447 - val_loss: 0.0029 - val_acc: 0.1134
Epoch 57/800
864/864 [==============================] - 6s 6ms/step - loss: 0.0030 - acc: 0.1644 - val_loss: 0.0037 - val_acc: 0.0825
Epoch 58/800
864/864 [==============================] - 6s 7ms/step - loss: 0.0031 - acc: 0.1505 - val_loss: 0.0031 - val_acc: 0.1443
Epoch 59/800
864/864 [==============================] - 5s 6ms/step - loss: 0.0031 - acc: 0.1632 - val_loss: 0.0038 - val_acc: 0.0825
Epoch 60/800
864/864 [==============================] - 6s 6ms/step - loss: 0.0032 - acc: 0.2130 - val_loss: 0.0032 - val_acc: 0.1340
Epoch 61/800
864/864 [==============================] - 6s 7ms/step - loss: 0.0029 - acc: 0.1308 - val_loss: 0.0033 - val_acc: 0.1443
Epoch 62/800
864/864 [==============================] - 6s 6ms/step - loss: 0.0029 - acc: 0.1713 - val_loss: 0.0027 - val_acc: 0.0928
Epoch 63/800
864/864 [==============================] - 6s 6ms/step - loss:

864/864 [==============================] - 9s 11ms/step - loss: 0.0021 - acc: 0.1123 - val_loss: 0.0034 - val_acc: 0.0722
Epoch 118/800
864/864 [==============================] - 8s 9ms/step - loss: 0.0023 - acc: 0.1863 - val_loss: 0.0039 - val_acc: 0.2784
Epoch 119/800
864/864 [==============================] - 8s 9ms/step - loss: 0.0022 - acc: 0.1806 - val_loss: 0.0041 - val_acc: 0.0825
Epoch 120/800
864/864 [==============================] - 8s 9ms/step - loss: 0.0022 - acc: 0.1331 - val_loss: 0.0039 - val_acc: 0.0722
Epoch 121/800
864/864 [==============================] - 8s 9ms/step - loss: 0.0021 - acc: 0.1771 - val_loss: 0.0039 - val_acc: 0.2680
Epoch 122/800
864/864 [==============================] - 8s 9ms/step - loss: 0.0023 - acc: 0.1134 - val_loss: 0.0035 - val_acc: 0.0722
Epoch 123/800
864/864 [==============================] - 8s 10ms/step - loss: 0.0022 - acc: 0.1736 - val_loss: 0.0039 - val_acc: 0.2887
Epoch 124/800
864/864 [==============================] - 8s 9ms/ste

864/864 [==============================] - 12s 13ms/step - loss: 0.0017 - acc: 0.1655 - val_loss: 0.0046 - val_acc: 0.0619
Epoch 178/800
864/864 [==============================] - 12s 13ms/step - loss: 0.0018 - acc: 0.1782 - val_loss: 0.0043 - val_acc: 0.2371
Epoch 179/800
864/864 [==============================] - 11s 13ms/step - loss: 0.0021 - acc: 0.1898 - val_loss: 0.0045 - val_acc: 0.2990
Epoch 180/800
864/864 [==============================] - 12s 14ms/step - loss: 0.0017 - acc: 0.1609 - val_loss: 0.0050 - val_acc: 0.1134
Epoch 181/800
864/864 [==============================] - 12s 14ms/step - loss: 0.0016 - acc: 0.1748 - val_loss: 0.0042 - val_acc: 0.2680
Epoch 182/800
864/864 [==============================] - 11s 13ms/step - loss: 0.0016 - acc: 0.2025 - val_loss: 0.0044 - val_acc: 0.2990
Epoch 183/800
864/864 [==============================] - 11s 13ms/step - loss: 0.0015 - acc: 0.1701 - val_loss: 0.0038 - val_acc: 0.2268
Epoch 184/800
864/864 [==============================] 

Epoch 237/800
864/864 [==============================] - 15s 17ms/step - loss: 0.0012 - acc: 0.1597 - val_loss: 0.0042 - val_acc: 0.0928
Epoch 238/800
864/864 [==============================] - 16s 18ms/step - loss: 0.0012 - acc: 0.1678 - val_loss: 0.0048 - val_acc: 0.1753
Epoch 239/800
864/864 [==============================] - 14s 17ms/step - loss: 0.0012 - acc: 0.1620 - val_loss: 0.0046 - val_acc: 0.0722
Epoch 240/800
864/864 [==============================] - 14s 16ms/step - loss: 0.0011 - acc: 0.1806 - val_loss: 0.0046 - val_acc: 0.1031
Epoch 241/800
864/864 [==============================] - 14s 16ms/step - loss: 0.0012 - acc: 0.1771 - val_loss: 0.0043 - val_acc: 0.2577
Epoch 242/800
864/864 [==============================] - 14s 16ms/step - loss: 0.0011 - acc: 0.1644 - val_loss: 0.0045 - val_acc: 0.1134
Epoch 243/800
864/864 [==============================] - 14s 17ms/step - loss: 0.0011 - acc: 0.1840 - val_loss: 0.0048 - val_acc: 0.1340
Epoch 244/800
864/864 [==================

864/864 [==============================] - 17s 20ms/step - loss: 8.8976e-04 - acc: 0.2257 - val_loss: 0.0048 - val_acc: 0.1340
Epoch 297/800
864/864 [==============================] - 17s 20ms/step - loss: 8.8232e-04 - acc: 0.2326 - val_loss: 0.0049 - val_acc: 0.2165
Epoch 298/800
864/864 [==============================] - 17s 20ms/step - loss: 0.0011 - acc: 0.2188 - val_loss: 0.0060 - val_acc: 0.2268
Epoch 299/800
864/864 [==============================] - 17s 20ms/step - loss: 0.0011 - acc: 0.2095 - val_loss: 0.0040 - val_acc: 0.0722
Epoch 300/800
864/864 [==============================] - 17s 20ms/step - loss: 9.8916e-04 - acc: 0.2130 - val_loss: 0.0058 - val_acc: 0.0619
Epoch 301/800
864/864 [==============================] - 18s 20ms/step - loss: 9.4931e-04 - acc: 0.2303 - val_loss: 0.0053 - val_acc: 0.1649
Epoch 302/800
864/864 [==============================] - 17s 20ms/step - loss: 9.3368e-04 - acc: 0.2095 - val_loss: 0.0053 - val_acc: 0.0619
Epoch 303/800
864/864 [============

In [ ]:
visualize_training_results(res)

In [ ]:
plt.figure(figsize=(12,5))

# Getting predictions by predicting from the last available X variable
yhat = model.predict(X[-1].reshape(1, n_per_in, n_features)).tolist()[0]

# Transforming values back to their normal prices
yhat = scaler.inverse_transform(np.array(yhat).reshape(-1,1)).tolist()

# Getting the actual values from the last available y variable which correspond to its respective X variable
actual = scaler.inverse_transform(y[-1].reshape(-1,1))

# Printing and plotting those predictions
print("Predicted Prices:\n", yhat)
plt.plot(yhat, label='Predicted')

# Printing and plotting the actual values
print("\nActual Prices:\n", actual.tolist())
plt.plot(actual.tolist(), label='Actual')

plt.title(f"Predicted vs Actual Closing Prices")
plt.ylabel("Price")
plt.legend()
plt.savefig("BTC_validation.png")
plt.show()

In [ ]:

# Predicting off of y because it contains the most recent dates
yhat = model.predict(np.array(df.tail(n_per_in)).reshape(1, n_per_in, n_features)).tolist()[0]

# Transforming the predicted values back to their original prices
yhat = scaler.inverse_transform(np.array(yhat).reshape(-1,1)).tolist()

# Creating a DF of the predicted prices
preds = pd.DataFrame(yhat, index=pd.date_range(start=df.index[-1], periods=len(yhat), freq="D"), columns=df.columns)

# Printing the predicted prices
print(preds)

# Number of periods back to visualize the actual values
pers = 10

# Transforming the actual values to their original price
actual = pd.DataFrame(scaler.inverse_transform(df[["Close"]].tail(pers)), index=df.Close.tail(pers).index, columns=df.columns).append(preds.head(1))

# Plotting
plt.figure(figsize=(16,6))
plt.plot(actual, label="Actual Prices")
plt.plot(preds, label="Predicted Prices")
plt.ylabel("Price")
plt.xlabel("Dates")
plt.title(f"Forecasting the next {len(yhat)} days")
plt.legend()
plt.savefig("BTC_predictions.png")
plt.show()